### In this file, we will conduct all of our tests

In [2]:
!pip install nbimporter

In [3]:
import pandas as pd
import numpy as np
import torch
from dgl.dataloading.pytorch import GraphDataLoader
from tqdm.notebook import tqdm
import nbimporter
import Data_Structs as ds
import model as mfile
from score import test

In [4]:
# sample code from https://docs.dgl.ai/tutorials/blitz/3_message_passing.html#sphx-glr-tutorials-blitz-3-message-passing-py

In [5]:
# def train(g, model):
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#     all_logits = []
#     best_val_acc = 0
#     best_test_acc = 0

#     features = g.ndata['feat']
#     labels = g.ndata['label']
#     train_mask = g.ndata['train_mask']
#     val_mask = g.ndata['val_mask']
#     test_mask = g.ndata['test_mask']
#     for e in range(200):
#         # Forward
#         logits = model(g, features)

#         # Compute prediction
#         pred = logits.argmax(1)

#         # Compute loss
#         # Note that we should only compute the losses of the nodes in the training set,
#         # i.e. with train_mask 1.
#         loss = F.cross_entropy(logits[train_mask], labels[train_mask])

#         # Compute accuracy on training/validation/test
#         train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
#         val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
#         test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

#         # Save the best validation accuracy and the corresponding test accuracy.
#         if best_val_acc < val_acc:
#             best_val_acc = val_acc
#             best_test_acc = test_acc

#         # Backward
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         all_logits.append(logits.detach())

#         if e % 5 == 0:
#             print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
#                 e, loss, val_acc, best_val_acc, test_acc, best_test_acc))



In [31]:
train_dataset = ds.SyntheticDataset()
batch_size = 16
train_dataloader = GraphDataLoader(train_dataset, batch_size = batch_size, shuffle = True)

In [51]:
from sklearn.metrics import mean_absolute_error as MAE

def train(model, epochs, output=False, debug_batch_interval=5):
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    
    for epoch in tqdm(range(epochs)):
        
        running, ct = 0, 0
        print('Epoch', epoch)
        for batch_idx, (batch_x, batch_y) in enumerate(train_dataloader):
            optimizer.zero_grad()
            
            bf = batch_x.edata['bond_feats'].float()
            af = batch_x.ndata['atom_feats'].float()
            y_pred = model(batch_x, af, bf)
            
            # The 23.06 is the same value used in score.py (conversion to kcal/mol)
            print("y_pred shape", y_pred.shape)
            print("y_pred", y_pred)
            print("batch_y", batch_y)
            loss = MAE(y_pred, batch_y) * 23.06 # ((y_pred.reshape(1,-1) - batch_y)**2).sum()
            running += loss.item()
            ct += 1
            loss.backward()
            optimizer.step()
            
            # Every debug_batch_interval iterations, print the data we've churned through (iterations * data per batch)
            if output and batch_idx % (len(train_dataloader) // debug_batch_interval) == 0:                
                print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.0f}'.format(
                          epoch, batch_idx * len(batch_x), len(train_dataloader) * batch_size,
                          100. * batch_idx / len(train_dataloader), loss.item()))
                
        if output:
            print("\nAverage Loss:", running / ct,"\n")

#### Create and Train Model

In [52]:
# All graphs in the list have the same scheme size, so pull the dimensions from the first
node_dim = train_dataset[0][0].ndata['atom_feats'].shape[1]
edge_dim = train_dataset[0][0].edata['bond_feats'].shape[1]
print("Dimensions:", node_dim, "(node),", edge_dim, "(edge)")

Dimensions: 11 (node), 5 (edge)


In [53]:
model = mfile.MPNN_Rec(node_dim, edge_dim, out_dim=1)

In [54]:
train(model, 10, output=True)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0
y_pred shape torch.Size([])
y_pred tensor(0.3070, grad_fn=<MeanBackward0>)
batch_y tensor([-2097.3260, -2702.4592, -4240.1252, -2698.1332, -2652.0016, -2709.6368,
        -2949.0479, -4280.7270, -2760.2251, -3748.6423, -2612.9286, -2452.0599,
        -2467.6564, -3342.5018, -2380.0580, -4434.0135], dtype=torch.float64)


TypeError: Singleton array tensor(0.3070, grad_fn=<MeanBackward0>) cannot be considered a valid collection.

In [ ]:
# Min loss = 89,224,320.50206168
# Min loss = 81,948,332